In [65]:
import import_ipynb
from Job import Job
from Vehicle import Vehicle
import importlib
import numpy as np
from typing import Dict, Tuple, Sequence, List, Any
import matplotlib.pyplot as plt
import folium
import pandas as pd 
import datetime

import json

In [80]:
from datetime import datetime

df = pd.read_csv("processed_dhl_data.csv")


# morning_parcel = "01/15/2020 00:00"
# until_morning_parcel = "01/16/2020 09:15"

# afternoon_parcel = "01/16/2020 09:16"
# until_afternoon_parcel = "01/16/2020 1:15"

morning_parcel = "2020/1/15 00:00"
until_morning_parcel = "2020/1/16 09:15"

afternoon_parcel = "2020/1/15 09:16"
until_afternoon_parcel = "2020/1/16 1:15"

# morning_parcel_dt = datetime.strptime(morning_parcel, "%m/%d/%Y %H:%M")
# until_morning_parcel_dt = datetime.strptime(until_morning_parcel, "%m/%d/%Y %H:%M")
# afternoon_parcel_dt = datetime.strptime(afternoon_parcel, "%m/%d/%Y %H:%M")
# until_afternoon_parcel_dt = datetime.strptime(until_afternoon_parcel, "%m/%d/%Y %H:%M")

#df = df.loc[datetime.strptime(df["AR dtm"], "%m/%d/%Y %H:%M")  ]

delivery = df.loc[df['Act Base'] == "D"]
pickup = df.loc[df['Act Base'] == "P"]

#no arrival time and delivery task can not be use
delivery = df.dropna(subset = ['AR dtm'])


delivery['AR dtm'] =  pd.to_datetime(delivery['AR dtm'], format='%m/%d/%Y %H:%M')

mask = (delivery['AR dtm'] >= morning_parcel) & (delivery['AR dtm'] <= until_morning_parcel)
dMorning = delivery.loc[mask]
dAfternoon = delivery.loc[~mask]

pickupDelivery = delivery.append(pickup, ignore_index=True)

pickupDeliveryMorning = dMorning.append(pickup, ignore_index=True)
pickupDeliveryAfternoon = dAfternoon.append(pickup, ignore_index=True)

courierName = df["Courier id"].unique()
zip = df["zip"].unique()
vehiclesNumber = len(courierName)

#afternoon
zip.astype(int)
pickupDeliveryAfternoon
#dMorning.to_csv("morning_data.csv")

D:\Users\MX-15\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,PUD Rte,PUD Cycle,Courier id,zip,Act Base,Prod Grp,Prod Code,AR dtm,Act Tm,Open,Closed,lat,lgtd,Act Ckpt Code,PuD Type
0,40,PJA1,B,goramali,40460,D,WPX,P,2019-12-25 05:33:00,15:53,NaN,23:59,2.988172,101.503285,OK,DL
1,53,PJA1,B,goramali,40460,D,WPX,P,2020-01-14 01:03:00,17:05,NaN,23:59,2.981229,101.514745,OK,DL
2,71,PJA2,B,mbinmuha,40460,D,WPX,P,2020-01-16 09:19:00,10:58,NaN,23:59,3.000842,101.533210,OK,DL
3,101,PJA2,B,mbinmuha,40460,D,WPX,P,2020-01-16 09:19:00,12:48,NaN,23:59,2.989479,101.546412,OK,DL
4,113,PJA2,B,mbinmuha,40460,D,DOM,N,2020-01-16 13:08:00,14:39,NaN,23:59,3.004487,101.543418,OK,DL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,3933,PJR1,B,fazwahab,47301,P,NaN,NaN,NaN,17:15,12:45,16:30,3.113009,101.576075,FP,FP
235,3935,PJR1,B,fazwahab,47301,P,NaN,NaN,NaN,17:15,16:30,17:30,3.113009,101.576075,FP,FP
236,4074,PJR4,B,tellappa,47000,P,NaN,NaN,NaN,16:25,13:00,16:30,3.228113,101.567032,FP,FP
237,4079,PJR4,B,tellappa,47000,P,NaN,NaN,NaN,17:22,17:00,17:30,3.208835,101.568147,FP,FP


In [67]:
vehicleList = []

for x in range(0, vehiclesNumber):
    vehicleList.append(Vehicle(x+1,[101.5531605, 3.0404532000000004],[99999], zip , [0, 14400]))    

# for x in range(0, vehiclesNumber):
#     vehicleList.append(Vehicle(x + len(vehiclesNumber) + 1,[101.5531605, 3.0404532000000004],[99999], zip , [21600, 36000]))  

In [68]:
vehicleList[0].id

1

In [102]:
jobList = []
#test = Job(1,[1],[1.98935, 48.701],[1], [[32400, 36000]])
#len(pickupDeliveryMorning)
for x in range(0, len(pickupDeliveryMorning)):
    jobList.append(Job(x+1,[1],[pickupDeliveryMorning.iloc[x]["lgtd"], pickupDeliveryMorning.iloc[x]["lat"]] , [pickupDeliveryMorning.iloc[x]["zip"]], [] ))

In [103]:
jobList[0].location

[101.52341390000001, 3.0171031000000004]

In [104]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

In [105]:
test = []
dict_t = {}
dict_t["vehicles"] = []
dict_t["jobs"] = []
for x in vehicleList:
    #test.append(json.dumps(x.__dict__))
    dict_t["vehicles"].append(x.__dict__)
    
for x in jobList:
    dict_t["jobs"].append(x.__dict__)
dict_t

{'vehicles': [{'id': 1,
   'start': [101.5531605, 3.0404532000000004],
   'capacity': [99999],
   'time_window': [0, 14400]},
  {'id': 2,
   'start': [101.5531605, 3.0404532000000004],
   'capacity': [99999],
   'time_window': [0, 14400]},
  {'id': 3,
   'start': [101.5531605, 3.0404532000000004],
   'capacity': [99999],
   'time_window': [0, 14400]},
  {'id': 4,
   'start': [101.5531605, 3.0404532000000004],
   'capacity': [99999],
   'time_window': [0, 14400]},
  {'id': 5,
   'start': [101.5531605, 3.0404532000000004],
   'capacity': [99999],
   'time_window': [0, 14400]},
  {'id': 6,
   'start': [101.5531605, 3.0404532000000004],
   'capacity': [99999],
   'time_window': [0, 14400]},
  {'id': 7,
   'start': [101.5531605, 3.0404532000000004],
   'capacity': [99999],
   'time_window': [0, 14400]},
  {'id': 8,
   'start': [101.5531605, 3.0404532000000004],
   'capacity': [99999],
   'time_window': [0, 14400]},
  {'id': 9,
   'start': [101.5531605, 3.0404532000000004],
   'capacity': [9

In [106]:
a = json.dumps(dict_t, cls=NpEncoder) 

text_file = open("morning.json", "w")
n = text_file.write(a )
text_file.close()

#json.dumps(dict_t)

In [107]:
a

'{"vehicles": [{"id": 1, "start": [101.5531605, 3.0404532000000004], "capacity": [99999], "time_window": [0, 14400]}, {"id": 2, "start": [101.5531605, 3.0404532000000004], "capacity": [99999], "time_window": [0, 14400]}, {"id": 3, "start": [101.5531605, 3.0404532000000004], "capacity": [99999], "time_window": [0, 14400]}, {"id": 4, "start": [101.5531605, 3.0404532000000004], "capacity": [99999], "time_window": [0, 14400]}, {"id": 5, "start": [101.5531605, 3.0404532000000004], "capacity": [99999], "time_window": [0, 14400]}, {"id": 6, "start": [101.5531605, 3.0404532000000004], "capacity": [99999], "time_window": [0, 14400]}, {"id": 7, "start": [101.5531605, 3.0404532000000004], "capacity": [99999], "time_window": [0, 14400]}, {"id": 8, "start": [101.5531605, 3.0404532000000004], "capacity": [99999], "time_window": [0, 14400]}, {"id": 9, "start": [101.5531605, 3.0404532000000004], "capacity": [99999], "time_window": [0, 14400]}, {"id": 10, "start": [101.5531605, 3.0404532000000004], "ca

In [148]:
import requests
import json
import pprint

url = 'http://35.213.166.175:3000'
#payload = open("request.json")
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
r = requests.post(url, data=open('morning.json', 'rb'), headers=headers)

In [124]:

r

<Response [200]>

In [125]:
pprint.pprint(json.loads(r.content))


{'code': 0,
 'routes': [{'amount': [32],
             'cost': 4082,
             'delivery': [32],
             'duration': 4082,
             'pickup': [0],
             'service': 9600,
             'steps': [{'arrival': 0,
                        'duration': 0,
                        'load': [32],
                        'location': [101.5531605, 3.0404532000000004],
                        'type': 'start'},
                       {'arrival': 107,
                        'duration': 107,
                        'job': 164,
                        'load': [31],
                        'location': [101.5535544, 3.0437508],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 516,
                        'duration': 216,
                        'job': 165,
                        'load': [30],
                        'location': [101.5555501, 3.0451416],
                        'se

                        'waiting_time': 0},
                       {'arrival': 3827,
                        'duration': 1727,
                        'job': 144,
                        'load': [25],
                        'location': [101.5620202, 3.0680789],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 4176,
                        'duration': 1776,
                        'job': 170,
                        'load': [24],
                        'location': [101.5599236, 3.068133],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 4799,
                        'duration': 2099,
                        'job': 173,
                        'load': [23],
                        'location': [101.5673634, 3.0739919],
                        'service': 300,
              

                        'duration': 1478,
                        'job': 678,
                        'load': [21],
                        'location': [101.5947246, 3.0360895],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 6457,
                        'duration': 1657,
                        'job': 671,
                        'load': [20],
                        'location': [101.5998313, 3.0428245],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 6836,
                        'duration': 1736,
                        'job': 670,
                        'load': [19],
                        'location': [101.6016355, 3.0421145],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
               

                        'waiting_time': 0},
                       {'arrival': 8445,
                        'duration': 3345,
                        'job': 223,
                        'load': [16],
                        'location': [101.4983958, 3.0562939],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 8860,
                        'duration': 3460,
                        'job': 226,
                        'load': [15],
                        'location': [101.4965819, 3.0558075],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 9265,
                        'duration': 3565,
                        'job': 231,
                        'load': [14],
                        'location': [101.4919451, 3.0583982999999995],
                        'service': 300,
    

                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 4536,
                        'duration': 1236,
                        'job': 625,
                        'load': [25],
                        'location': [101.5969791, 3.0643982000000003],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 4893,
                        'duration': 1293,
                        'job': 602,
                        'load': [24],
                        'location': [101.5958493, 3.0653824],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 5240,
                        'duration': 1340,
                        'job': 604,
                        'load': [23],
                        'location': [101.5930717, 3.0624535],
     

...

                        'duration': 4022,
                        'job': 1739,
                        'load': [7],
                        'location': [101.3728155, 3.0406345],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 11324,
                        'duration': 4124,
                        'job': 1736,
                        'load': [6],
                        'location': [101.378023, 3.0421614],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 11625,
                        'duration': 4125,
                        'job': 1737,
                        'load': [5],
                        'location': [101.3783182, 3.0425882],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
              

                        'duration': 3668,
                        'job': 2239,
                        'load': [13],
                        'location': [101.6257412, 3.1857637000000003],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 8830,
                        'duration': 3730,
                        'job': 2248,
                        'load': [12],
                        'location': [101.6242839, 3.1842991],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 9281,
                        'duration': 3881,
                        'job': 2240,
                        'load': [11],
                        'location': [101.6278324, 3.1841596],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
   

             'steps': [{'arrival': 0,
                        'duration': 0,
                        'load': [32],
                        'location': [101.5531605, 3.0404532000000004],
                        'type': 'start'},
                       {'arrival': 1543,
                        'duration': 1543,
                        'job': 2232,
                        'load': [31],
                        'location': [101.6584289, 3.1770080000000003],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 2181,
                        'duration': 1881,
                        'job': 2093,
                        'load': [30],
                        'location': [101.6709169, 3.2245283],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 2620,
                        'duration': 

                        'job': 2608,
                        'load': [20],
                        'location': [101.5601685, 3.2208132000000003],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 5529,
                        'duration': 2829,
                        'job': 2606,
                        'load': [19],
                        'location': [101.5582756, 3.2191054],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 6155,
                        'duration': 3155,
                        'job': 2616,
                        'load': [18],
                        'location': [101.5643716, 3.2353327000000003],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 6

                        'location': [101.8262199, 2.9629182999999997],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 7903,
                        'duration': 4603,
                        'job': 2056,
                        'load': [12],
                        'location': [101.8137929, 2.9609263],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 8266,
                        'duration': 4666,
                        'job': 2040,
                        'load': [11],
                        'location': [101.8121373, 2.9624636000000004],
                        'service': 300,
                        'type': 'job',
                        'waiting_time': 0},
                       {'arrival': 8673,
                        'duration': 4773,
                        'job

In [149]:
test = json.loads(r.content)

In [157]:
unassigned_jobs = test['unassigned']
unassigned_jobs[0]['location']

101.5857766

In [161]:
jobList = []
#test = Job(1,[1],[1.98935, 48.701],[1], [[32400, 36000]])
#len(pickupDeliveryMorning)
for x in range(0, len(unassigned_jobs)):
    jobList.append(Job(unassigned_jobs[x]['id'],[1],[unassigned_jobs[x]["location"][0], unassigned_jobs[x]["location"][1]] , [111], [] ))    

for x in range(0, len(pickupDeliveryAfternoon)):
    jobList.append(Job(x+3000,[1],[pickupDeliveryAfternoon.iloc[x]["lgtd"], pickupDeliveryAfternoon.iloc[x]["lat"]] , [pickupDeliveryAfternoon.iloc[x]["zip"]], [] ))

test = []
dict_t = {}
dict_t["vehicles"] = []
dict_t["jobs"] = []
for x in vehicleList:
    #test.append(json.dumps(x.__dict__))
    dict_t["vehicles"].append(x.__dict__)
    
for x in jobList:
    dict_t["jobs"].append(x.__dict__)
    
a = json.dumps(dict_t, cls=NpEncoder) 

text_file = open("afternoon.json", "w")
n = text_file.write(a )
text_file.close()

import requests
import json
import pprint

url = 'http://35.213.166.175:3000'
#payload = open("request.json")
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
r = requests.post(url, data=open('afternoon.json', 'rb'), headers=headers)


In [162]:
r

<Response [200]>

In [163]:
r.content

b'{"code":0,"summary":{"cost":81201,"unassigned":0,"delivery":[258],"amount":[258],"pickup":[0],"service":77400,"duration":81201,"waiting_time":0,"computing_times":{"loading":225,"solving":1646}},"unassigned":[],"routes":[{"vehicle":1,"cost":8932,"delivery":[18],"amount":[18],"pickup":[0],"service":5400,"duration":8932,"waiting_time":0,"steps":[{"type":"start","location":[101.5531605,3.0404532000000006],"load":[18],"arrival":0,"duration":0},{"type":"job","location":[101.545155,3.0564612999999998],"job":3073,"service":300,"waiting_time":0,"load":[17],"arrival":343,"duration":343},{"type":"job","location":[101.5534596,3.3097056000000006],"job":3238,"service":300,"waiting_time":0,"load":[16],"arrival":2458,"duration":2158},{"type":"job","location":[101.5533901,3.3096137999999999],"job":3140,"service":300,"waiting_time":0,"load":[15],"arrival":2761,"duration":2161},{"type":"job","location":[101.5666473,3.3047282999999996],"job":2522,"service":300,"waiting_time":0,"load":[14],"arrival":3517